In [12]:
import sys

!conda install -c conda-forge geopy --yes 

print('Packages installed.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\kmohammad\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0
  certifi                      

In [14]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import scipy
# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html.parser') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.') 

Page Scrapped.


In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [6]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
        #g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        #lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [7]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75242,-79.329242
1,M4A,North York,Victoria Village,43.75242,-79.329242
2,M5A,Downtown Toronto,Harbourfront,43.75242,-79.329242
3,M6A,North York,Lawrence Heights,43.75242,-79.329242
4,M6A,North York,Lawrence Manor,43.75242,-79.329242
...,...,...,...,...,...
204,M8Y,Etobicoke,Sunnylea,43.75242,-79.329242
205,M8Z,Etobicoke,Kingsway Park South West\n,43.75242,-79.329242
206,M8Z,Etobicoke,Mimico NW,43.75242,-79.329242
207,M8Z,Etobicoke,The Queensway West,43.75242,-79.329242


In [8]:
neighbors.shape

(209, 5)

### Lets Explore the area

In [15]:
CLIENT_ID = 'XSLPNWWA0WBALTH2KD5FZMDAP03CMN3AJLTAKX13G4GO' # your Foursquare ID
CLIENT_SECRET = 'MVTCFRSYI5ADIJQBKZVWQFV3MCHCVM2RVB3QHE1F3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XSLPNWWA0WBALTH2KD50LH4FZMDAP03CMN3AJLTAKX13G4GO
CLIENT_SECRET:MVTCFRSYI5AUFWDGTJDIJQBKZVWQFV3MCHCVM2RVB3QHE1F3


In [22]:


VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

#search_query = 'Italian'

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


{'meta': {'code': 200, 'requestId': '5e39457047e0d6001b5f4f98'},
 'response': {'venues': [{'id': '4e42684718a8627fce453c01',
    'name': 'TTC stop #8380',
    'location': {'address': 'Underhill Dr',
     'crossStreet': 'At Cassandra N',
     'lat': 43.752672,
     'lng': -79.326351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.752672,
       'lng': -79.326351}],
     'distance': 234,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Underhill Dr (At Cassandra N)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b4f',
      'name': 'Bus Stop',
      'pluralName': 'Bus Stops',
      'shortName': 'Bus Stop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580811683',
    'hasPerk': False},
   {'id': '4e8d9dcdd5fbbbb6b3003c7b',
    'name': 'Brookbanks Park',
    '

In [26]:
from pandas.io.json import json_normalize
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,venuePage.id
0,4e42684718a8627fce453c01,TTC stop #8380,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1580811683,False,Underhill Dr,At Cassandra N,43.752672,-79.326351,"[{'label': 'display', 'lat': 43.752672, 'lng':...",234,CA,Toronto,ON,Canada,"[Underhill Dr (At Cassandra N), Toronto ON, Ca...",NaN,NaN,NaN
1,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1580811683,False,Toronto,NaN,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",238,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]",NaN,NaN,NaN
2,5927692fb8fd9d01a7707ee1,Brunswick Bierworks,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1580811683,False,25 Curity Ave,NaN,43.710667,-79.317720,"[{'label': 'display', 'lat': 43.710667, 'lng':...",4739,CA,East York,ON,Canada,"[25 Curity Ave, East York ON M4B 3M2, Canada]",M4B 3M2,O'Connor - Parkview,NaN
3,535fddb1498e03814e03968f,GTA Restoration | Emergency Water Damage Plumb...,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",v-1580811683,False,250 Yonge St,401 & DVP,43.753567,-79.351308,"[{'label': 'display', 'lat': 43.7535666482373,...",1778,CA,Toronto,ON,Canada,"[250 Yonge St (401 & DVP), Toronto ON M5B 2L7,...",M5B 2L7,NaN,84830578
4,4b8991cbf964a520814232e3,Allwyn's Bakery,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1580811683,False,81 Underhill drive,NaN,43.759840,-79.324719,"[{'label': 'display', 'lat': 43.75984035203157...",902,CA,Toronto,ON,Canada,"[81 Underhill drive, Toronto ON M3A 1Z5, Canada]",M3A 1Z5,Parkwoods - Donalda,NaN


In [27]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Get Nearby Venues

In [37]:
from pandas.io.json import json_normalize

#venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TTC stop #8380,Bus Stop,43.752672,-79.326351
1,Brookbanks Park,Park,43.751976,-79.332140
2,Brunswick Bierworks,Brewery,43.710667,-79.317720
3,GTA Restoration | Emergency Water Damage Plumb...,Construction & Landscaping,43.753567,-79.351308
4,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719


In [42]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=13) 


folium.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup=neighborhood_name,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Get Venue details of Allwyn's Bakery

In [43]:
venue_id = '4b8991cbf964a520814232e3' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4b8991cbf964a520814232e3?client_id=XSLPNWWA0WBALTH2KD50LH4FZMDAP03CMN3AJLTAKX13G4GO&client_secret=MVTCFRSYI5AUFWDGTJDIJQBKZVWQFV3MCHCVM2RVB3QHE1F3&v=20180605'

In [44]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b8991cbf964a520814232e3',
 'name': "Allwyn's Bakery",
 'contact': {'phone': '4163911757', 'formattedPhone': '(416) 391-1757'},
 'location': {'address': '81 Underhill drive',
  'lat': 43.75984035203157,
  'lng': -79.32471879917513,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.75984035203157,
    'lng': -79.32471879917513}],
  'postalCode': 'M3A 1Z5',
  'cc': 'CA',
  'neighborhood': 'Parkwoods - Donalda',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['81 Underhill drive', 'Toronto ON M3A 1Z5', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/allwyns-bakery/4b8991cbf964a520814232e3',
 'categories': [{'id': '4bf58dd8d48988d144941735',
   'name': 'Caribbean Restaurant',
   'pluralName': 'Caribbean Restaurants',
   'shortName': 'Caribbean',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 16},
 'price': {'tier':

## Lets Check rating

In [45]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.8


## Get Number of TIPS

In [46]:
result['response']['venue']['tips']['count']

16

In [47]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e39905e77af030028bc269d'},
 'response': {'tips': {'count': 16,
   'items': [{'id': '5764a3e2498eba30c2fb0ec1',
     'createdAt': 1466213346,
     'text': 'stuffed jerk chicken sandwich - Jamaican beef pattie inside a jerk chicken sandwich with delicious creamy coleslaw, really awesome.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5764a3e2498eba30c2fb0ec1',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '2049936',
      'firstName': 'Sara',
      'lastName': 'B',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/2049936-FE1ZB4JE34ZQFE2N.jpg'}},
     'authorInteractionType': 'liked'}]}}}

### Get Associated Feautres

In [49]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [50]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\kmohammad\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"stuffed jerk chicken sandwich - Jamaican beef pattie inside a jerk chicken sandwich with delicious creamy coleslaw, really awesome.",3,0,5764a3e2498eba30c2fb0ec1,Sara,B,NaN,2049936


## Lets Explore location at Allwyn's Bakery

In [51]:
latitude = 43.759840
longitude = -79.324719

In [52]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XSLPNWWA0WBALTH2KD50LH4FZMDAP03CMN3AJLTAKX13G4GO&client_secret=MVTCFRSYI5AUFWDGTJDIJQBKZVWQFV3MCHCVM2RVB3QHE1F3&ll=43.75984,-79.324719&v=20180605&radius=500&limit=100'

In [54]:
import requests
results = requests.get(url).json()
'There are {} around Allwyn\'s restaurant.'.format(len(results['response']['groups'][0]['items']))

"There are 13 around Allwyn's restaurant."

In [55]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b8991cbf964a520814232e3',
  'name': "Allwyn's Bakery",
  'location': {'address': '81 Underhill drive',
   'lat': 43.75984035203157,
   'lng': -79.32471879917513,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.75984035203157,
     'lng': -79.32471879917513}],
   'distance': 0,
   'postalCode': 'M3A 1Z5',
   'cc': 'CA',
   'neighborhood': 'Parkwoods - Donalda',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['81 Underhill drive', 'Toronto ON M3A 1Z5', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d144941735',
    'name': 'Caribbean Restaurant',
    'pluralName': 'Caribbean Restaurants',
    'shortName': 'Caribbean',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'group

In [56]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Allwyn's Bakery,Caribbean Restaurant,81 Underhill drive,43.759840,-79.324719,"[{'label': 'display', 'lat': 43.75984035203157, 'lng': -79.32471879917513}]",0,M3A 1Z5,CA,Parkwoods - Donalda,Toronto,ON,Canada,"[81 Underhill drive, Toronto ON M3A 1Z5, Canada]",NaN,4b8991cbf964a520814232e3
1,Tim Hortons,Café,215 Brookbanks,43.760668,-79.326368,"[{'label': 'display', 'lat': 43.76066827030228, 'lng': -79.32636763515987}]",161,M3A 1Z5,CA,NaN,Toronto,ON,Canada,"[215 Brookbanks (York Miils Rd), Toronto ON M3A 1Z5, Canada]",York Miils Rd,57e286f2498e43d84d92d34a
2,A&W,Fast Food Restaurant,1277 York Mills Road,43.760643,-79.326865,"[{'label': 'display', 'lat': 43.760643076161315, 'lng': -79.3268649067081}]",194,M3A 1Z5,CA,NaN,Toronto,ON,Canada,"[1277 York Mills Road, Toronto ON M3A 1Z5, Canada]",NaN,58a8dcaa6119f47b9a94dc05
3,Food Basics,Supermarket,1277 York Mills Road,43.760865,-79.326015,"[{'label': 'display', 'lat': 43.76086487186778, 'lng': -79.326015195595}]",154,M3A 1Z5,CA,NaN,Toronto,ON,Canada,"[1277 York Mills Road (at Parkwoods Village Dr.), Toronto ON M3A 1Z5, Canada]",at Parkwoods Village Dr.,4b26adc3f964a5205f7f24e3
4,Shoppers Drug Mart,Pharmacy,1277 York Mills Rd,43.760857,-79.324961,"[{'label': 'display', 'lat': 43.76085733239677, 'lng': -79.32496051573521}]",114,M3A 1Z5,CA,NaN,Toronto,ON,Canada,"[1277 York Mills Rd (At Parkwoods Village Drive), Toronto ON M3A 1Z5, Canada]",At Parkwoods Village Drive,4c422e48e26920a1a4ad5fe7
5,Pizza Pizza,Pizza Place,"1277 York Mills Road, Unit 9",43.760231,-79.325666,"[{'label': 'display', 'lat': 43.76023098701761, 'lng': -79.32566642761229}]",87,M3A 1Z5,CA,NaN,North York,ON,Canada,"[1277 York Mills Road, Unit 9, North York ON M3A 1Z5, Canada]",NaN,4b5f7253f964a520d7ba29e3
6,Family Food Fair Convenience,Convenience Store,1277 York Mills Rd.,43.760620,-79.324459,"[{'label': 'display', 'lat': 43.760619989690674, 'lng': -79.32445934492416}]",89,NaN,CA,NaN,Toronto,ON,Canada,"[1277 York Mills Rd., Toronto ON, Canada]",NaN,4de41ec28877bcb6866ca329
7,Parkwoods Coin Laundry,Laundry Service,1277 York Mills Rd.,43.760386,-79.324894,"[{'label': 'display', 'lat': 43.76038596280906, 'lng': -79.32489395141602}]",62,M3A 1Z5,CA,NaN,North York,ON,Canada,"[1277 York Mills Rd., North York ON M3A 1Z5, Canada]",NaN,4c437dd1f0bdd13ad34ec8cc
8,Parkwoods Village Centre,Shopping Mall,1277 York Mills Rd,43.760735,-79.324873,"[{'label': 'display', 'lat': 43.760735, 'lng': -79.324873}]",100,NaN,CA,NaN,Toronto,ON,Canada,"[1277 York Mills Rd, Toronto ON, Canada]",NaN,4f5930bfe4b030058495a752
9,Dollarama,Discount Store,"1277 York Mills Rd,Parkwood Village",43.760341,-79.325519,"[{'label': 'display', 'lat': 43.760341, 'lng': -79.325519}]",85,M3A 1Z5,CA,Parkwoods - Donalda,North York,ON,Canada,"[1277 York Mills Rd,Parkwood Village, North York ON M3A 1Z5, Canada]",NaN,55bbdfb9498e5996dd9d4038


## Lets Visualize in Map

In [58]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Allwyn


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Allwyn\'s Bakery',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map